In [ ]:
!apt-get update 
!pip install stable_baselines3[extra] git+https://github.com/wandb/client.git
!apt-get install x11-utils > /dev/null 2>&1 
!!pip install pyglet==1.5.0
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install pyvirtualdisplay
!pip install pybullet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
import gym
import pybullet_envs
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
import wandb
from wandb.integration.sb3 import WandbCallback

display = Display(visible=0, size=(400, 300))
display.start()

In [ ]:
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 1_000_000,
    "env_name": "AntBulletEnv-v0",
}
run = wandb.init(
    project="sb3",
    name='ANT_1M_steps_param_1',
    config=config,
    sync_tensorboard=True,
    monitor_gym=True, 
    save_code=True,
)

def make_env():
    env = gym.make(config["env_name"])
    env = Monitor(env)
    return env

env = DummyVecEnv([make_env])
env = VecVideoRecorder(env, "videos", record_video_trigger=lambda x: x % 6000 == 0, video_length=200)
model = PPO(config["policy_type"], env, verbose=1, n_steps=512, tensorboard_log=f"runs", learning_rate=0.0005, gamma=0.99)
model.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=10,
        model_save_path=f"models/{run.id}",
        verbose=2,
    ),
)


In [ ]:
!zip -r /content/videos_1M_param_1.zip /content/videos/

In [ ]:
eval_env = gym.make('AntBulletEnv-v0')
env = DummyVecEnv([make_env])
model.save("ppo_ant_1M")

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
episodes = 5

for ep in range(episodes):
    obs = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        env.render()        

In [ ]:
model = PPO.load("ppo_ant_1M")

In [ ]:
mean_reward, std_reward = evaluate_policy(model,eval_env, render=False, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

Save agent

Test